# SMILES to Neo4j
This notebook converts SMILES to a Neo4J graph and uses a following .csv as input, which consists of:
- study.id: The identifier of the study
- study.type: the type of study
- uuid: the molecule identifier
- smiles: the SMILES string of the molecule
- name: the molecule name
- is_parent: if this molecule is at the start of a pathway, this will be true
- source_mol.uuid: if the molecule is a product of a reaction, this is the identifier of the reactant molecule
</br>An example can be seen below:

|study.id|study.type|uuid|smiles|name|is_parent|source_mol.uuid|
|---|---|---|---|---|---|---|
|MtPth_Map_001|Tomato (fruits)|m142536|ClC=CCCl|1,3-dichloropropene (1,3-D)|TRUE|null|
|MtPth_Map_001|Tomato (fruits)|m415263|OCCCCl|3-chloro-1-propanol||m142536|
|...|...|...|...|...|...|...|

Unfortunately, the original dataset could not be included as it contains data that is allowed to be used, but not publicly shared by third parties. However, the raw data can be found here: https://zenodo.org/record/5525989#.Y4DR8BTMKUn </br>
To make enable running the code, a smaller SMILES dataset in the same format is constructed of open source metabolic pathways of https://www.pathbank.org/.

## Import of the necessary packages 

In [1]:
from __future__ import print_function
import os, glob
from pathlib import Path
import subprocess
import shutil
from rdkit import Chem
from rdkit.Chem import AllChem, rdmolops, Lipinski
import itertools
import numpy as np
from functions import *
import pandas as pd
from neo4j import GraphDatabase
import openbabel

## Connect with Neo4j
Using the official Neo4j tool, the code below lets you connect to a Neo4j database of choice
</br>This code is based on Neo4j's example: https://neo4j.com/docs/api/python-driver/current/

In [2]:
class Neo4jConnection:
    """
    This class establishes the connectio with Neo4j and enables storing and retreiving data with cypher queries. The code is based on the official Neo4j documentation:
    https://neo4j.com/docs/api/python-driver/current/
    """
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

Below, the neo4j username, password and URL can be given, note that you have the credentials to your own database, the ones that can be found below will not work </br> After this, the connection with the Neo4j database is made

In [3]:
# Give username, password and database URL
NEO4J_USER = 'neo4j'
NEO4J_PWD = 'metapath'
BOLT_URL = 'bolt://localhost:7687/'

# Establish connection with Neo4j
conn = Neo4jConnection(uri=BOLT_URL, user=NEO4J_USER, pwd=NEO4J_PWD)

## SMILES to cypher

In [4]:
# Read the csv file containing the SMILES and additional information
df = pd.read_csv('SMILES.csv', sep=';')

In [5]:
df.loc[df['study.id']=='MtPth_Map_001'].index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], dtype='int64')

Delete, if it is there, the existing 'MOL_to_cypher.cypher' file

In [6]:
if os.path.isfile('MOL_to_cypher.cypher') == True:
    os.remove('MOL_to_cypher.cypher')

### The main algorithm

The code below creates cypher queries and runs them in Neo4j, queries are created for the reaction, molecule, atom, bond and ring nodes together with their edges. This is done with the following steps for each molecule:</br>
1. Load the SMILES with RDKit, this is used to get the most features for the different nodes
2. Create the molecule node and its features
3. Convert to a MOL2 file with OpenBabel for additional atom features
4. Use the ringinfo function from the functions.py file to get ring information
5. If applicable, create the ring nodes and an edge with the molecule
6. Create the atom nodes and their edges with the molecule node, and the ring node(s) if this is applicable
7. Create the bond nodes and their edges with the molecule and atom nodes
8. Run the queries created above in the Neo4j database
9. Create all edges between the rings and their corresponding bonds, if applicable, and run it in Neo4j
10. If the molecule is a product, search the corresponding product and create a reaction node for them, then run in Neo4j

In [7]:
# Loop over all studies in the dataset
for study in df['study.id'].unique():
    
    # Loop over all molecules in the study
    for f in df.loc[df['study.id']==study].index.values.tolist():
        
        # Delete the existing cypher file
        if os.path.isfile('MOL_to_cypher.cypher') == True:
            os.remove('MOL_to_cypher.cypher')
        
        with open('MOL_to_cypher.cypher', 'w') as w:
            # Store the molecule name
            mol_name = str(df.iloc[f,0]+'_'+df.iloc[f,2])
            mol_name = mol_name.replace('-', '_')
        
            # Add RDkit format and conformer
            rdmol = Chem.MolFromSmiles(df.iloc[f,3])
        
            # Cypher: create the molecule
            com = f"MERGE ({mol_name}:Molecule {{id:'{mol_name}'}})\n"
            com1 = f"ON CREATE\n"
            m_name = df.iloc[f,4].replace('\'', '')
            com2 = f"SET {mol_name}.smiles = '{df.iloc[f,3]}', {mol_name}.Hdonors = {Chem.Lipinski.NumHDonors(rdmol)}, {mol_name}.name = '{m_name}', {mol_name}.studyid = '{study}'\n"
            w.write(com+com1+com2)  
        
            # Convert to MOL2
            n = df.iloc[f,3]
            m = f"{mol_name}.mol2"
            !obabel -:$n -omol2 -O1-MOL2/$m
        
            # Get features from MOL2 file
            mol2 = open('1-MOL2/'+ mol_name +'.mol2')
            mf = []
            m = 0
            
            # Looks at every line of the MOL2 file
            for n in mol2:
                feat = []
                
                # Ensures only the atom features are used
                if '@<TRIPOS>BOND' in n:
                    m = 0

                # Stores the computed charche and type of the atom in the mf list
                if m == 1:
                    feat.append(n[70:76])
                    feat.append(n[47:52].strip())
                    mf.append(feat)

                # Ensures only the atom features are used
                if '@<TRIPOS>ATOM' in n:
                    m = 1
                    
            # Get the information of all the rings using the ringinfo function from functions.py
            ringlst, asys, arings, rings = ringinfo(rdmol)
            
            # Cypher: create all the rings
            for ring in ringlst:
                rtype = ring[0][:ring[0].index("_")] # Get the ringtype

                com = f"MERGE ({ring[0]}:Ring {{id:'{mol_name}_{ring[0]}'}})\n" # Create the ring node
                com1 = f"ON CREATE\n"
                com2 = f"SET {ring[0]}.ring_type = '{rtype}', {ring[0]}.size = {ring[1]}\n" # Add the additional features
                com2 = com2.replace(f"ring_type = 'ring'", "ring_type = 'aliphatic'") # Replace ring by aliphatic
                com3 = f"MERGE ({mol_name}) -[:HAS_RING]-> ({ring[0]})\n" # Create edge between the molecule and ring nodes
                w.write(com+com1+com2+com3) # Write everything to the MOL_to_cypher file
        
            # Write cypher line for each atom
            for atom in rdmol.GetAtoms():
                idx = atom.GetIdx() # Get atom index
                at_name = mol_name + '_' + str(atom.GetIdx()) # Create atom name by adding the index to the mol name

                com = f"MERGE ({at_name}:Atom {{id:'{at_name}'}})\n" # Create the atom node
                
                # Add the additional atom features
                com1 = f"ON CREATE\n"
                com2 = f"""
                        SET {at_name}.symbol = '{atom.GetSymbol()}', {at_name}.degree = {atom.GetTotalDegree()}, {at_name}.charge = {mf[idx][0]}, {at_name}.atom_type = '{mf[idx][1]}', 
                        {at_name}.hybridization = '{atom.GetHybridization()}', {at_name}.valence = {atom.GetExplicitValence()}\n
                        """
                
                com3 = f"MERGE ({mol_name}) -[:HAS_ATOM]-> ({at_name})\n" # Create the edge between the molecule and atom nodes
                w.write(com+com1+com2+com3) # Write everything to the cypher file
                
                # For every ring in the molecule, check if the atom is in it, if so create a atom-ring edge
                for sys in asys:
                    if idx in asys[sys]:
                        w.write(f"MERGE ({sys}) -[:HAS_ATOM]-> ({at_name})\n")

                for r in arings:
                    if idx in arings[r]:
                        w.write(f"MERGE ({r}) -[:HAS_ATOM]-> ({at_name})\n")

                for r in rings:
                    if idx in rings[r]:
                        w.write(f"MERGE ({r}) -[:HAS_ATOM]-> ({at_name})\n")
            
            # Write cypher line for each bond
            for bond in rdmol.GetBonds():
                begin = bond.GetBeginAtomIdx() # Get the begin atom of the bond
                end = bond.GetEndAtomIdx() # Get the end atom of the bond
                bo_name = mol_name + '_' + str(begin) + '_' + str(end) # Create bond name
                
                # Get the bond type, if the bond is aromatic change it to aromatic
                btype = str.lower(str(bond.GetBondType()))
                if bond.GetIsAromatic() == True:
                    btype = 'aromatic'
            
                com = f"MERGE ({bo_name}:Bond {{id:'{bo_name}'}})\n" # Create bond node
            
                # Add additional bond features
                com1 = f"ON CREATE\n"
                com2 = f"SET {bo_name}.bond_type = '{btype}'\n"
                com3 = f"MERGE ({mol_name}) -[:HAS_BOND]-> ({bo_name})\n" # Create molecule-bond edge
                com4 = f"MERGE ({mol_name+'_'+str(begin)}) -[:BONDED_WITH]-> ({bo_name}) <-[:BONDED_WITH]- ({mol_name+'_'+str(end)})\n" # Create atom-bond edge

                w.write(com+com1+com2+com3+com4) # Write everything to cyper file
                    
            w.write(';\n') # Add to close cypher part of the specific molecule
            w.close()
        
        # Open the created cypher file and run it in Neo4j
        f = open('MOL_to_cypher.cypher', 'r')
        conn.query(f.read())
        f.close()

        os.remove('MOL_to_cypher.cypher') # Remove existing cypher file
        
        # Create cypher command to create an edge between the ring and its bonds
        com = f"MATCH (m:Ring)-[:HAS_ATOM]->(a1:Atom)-[:BONDED_WITH]->(b:Bond)<-[:BONDED_WITH]-(a2:Atom)<-[:HAS_ATOM]-(m)\n"
        com1 = f"MERGE (m)-[:HAS_BOND]->(b)\n"
        com2 = f";\n"
        # Run in Neo4j
        conn.query(com+com1+com2)

    
    # Add the reactions between the molecules of the study
    for i, f in df.loc[df['study.id']==study].iterrows():
        
        # If a molecule in the study is linked to another molecule (when it's not NaN):
        if isinstance(f['source_mol.uuid'], str):
            
            # Create reaction name
            name = f"{f['study.id']}_{f['source_mol.uuid']}_{f['uuid']}"
            name = name.replace('-', '_')

            # Create reactant name
            rname = str(f['study.id']+'_'+f['source_mol.uuid'])
            rname = rname.replace('-', '_')
            
            # Create product name
            pname = str(f['study.id']+'_'+f['uuid'])
            pname = pname.replace('-', '_')

            # If it exists, remove the cypher file
            if os.path.isfile('MOL_to_cypher.cypher') == True:
                os.remove('MOL_to_cypher.cypher')

            with open('MOL_to_cypher.cypher', 'w') as w:
                com = f"MERGE ({name}:Reaction{{name:'{name}'}})\n" # Create reaction node
                
                # Add additional reaction features
                com1 = f"ON CREATE\n"
                com2 = f"SET {name}.studyid = '{study}', {name}.study_type = '{f['study.type']}'\n"

                # Match create the molecule-reaction edges
                com3 = f"WITH {name}\n"
                com4 = f"MATCH (p:Molecule),(r:Molecule)\n"
                com5 = f"WHERE p.id = '{pname}' AND r.id = '{rname}'\n" # This only works with the format currently provided
                com6 = f"MERGE (r)-[:REACTS_IN]->({name})-[:PRODUCES]->(p)\n"
                com7 = f";\n"
                w.write(com+com1+com2+com3+com4+com5+com6+com7) # Write everything to the cypher file
                w.close()
                
                # Open the created cypher file and run it in Neo4j
                f = open('MOL_to_cypher.cypher', 'r')
                conn.query(f.read())
                f.close()
                    

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
